# Upload Structure

In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData
from aiida.orm.calculation import Calculation

import ase.io
from ase.lattice.cubic import FaceCenteredCubic
from ase.build import bulk


import numpy as np
import ipywidgets as ipw
from base64 import b64decode
from IPython.display import display, clear_output, Image
from fileupload import FileUploadWidget

import nglview

In [ ]:
atoms = None
structures = [("select structure",{"status":False})]

layout = ipw.Layout(width="400px")
style = {"description_width":"150px"}

viewer = nglview.NGLWidget()
clear_output()

In [ ]:
def refresh_structure_view():
    global viewer, atoms
    if hasattr(viewer, "component_0"):
        #viewer.clear_representations()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)

    viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
    viewer.add_unitcell()
    viewer.center()

## Step 1: Upload file

In [ ]:
def on_click_button(c):
    global atoms
    with store_out:
        try:
            atoms  = bulk(inp_elements.value, cubic=True)
        except:
            query_message.value = "Error"
            cell_params.value = "Error"
            return
        cell_params.value = "Unit cell <br />a = {} {} {} <br />b = {} {} {} <br />c = {} {} {}".format(
            atoms.cell[0][0], atoms.cell[0][1], atoms.cell[0][2],
            atoms.cell[1][0], atoms.cell[1][1], atoms.cell[1][2],
            atoms.cell[2][0], atoms.cell[2][1], atoms.cell[2][2])
        query_message.value = "Ta-da"
        formula = atoms.get_chemical_formula()
        refresh_structure_view()


inp_elements = ipw.Text(description="", value="Cu", placeholder='e.g.: Ni', layout=layout, style=style)
btn_query = ipw.Button(description='Select element')
btn_query.on_click(on_click_button)
query_message = ipw.HTML("You did not select any element")
cell_params = ipw.HTML("Hello")
store_out = ipw.Output()



display(ipw.HBox([btn_query, inp_elements]), query_message, cell_params, ipw.VBox([viewer]))

In [ ]:
def on_click_store(b):
    global atoms
    with store_out:
        clear_output()
        #AiiDA requires structures to have cell
        if np.all(atoms.cell == 0.0):
            atoms.center(vacuum=0.1)
        s = StructureData(ase=atoms)
        # ensure that tags got correctly translated into kinds 
        for t1, k in zip(atoms.get_tags(), s.get_site_kindnames()):
            t2 = int(k[-1]) if k[-1].isnumeric() else 0
            assert t1==t2
        s.description = inp_descr.value
        s.store()
        print("Stored in AiiDA: "+repr(s))

inp_descr = ipw.Text(placeholder="Description (optional)")   
btn_store = ipw.Button(description='Store in AiiDA')
btn_store.on_click(on_click_store)
store_out = ipw.Output()
display(ipw.HBox([btn_store, inp_descr]), store_out)

## Step 1: Optimize the unit cell

In [ ]:
from aiida.orm import load_node, Code
from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.upf import get_pseudos_from_structure
from aiida.orm.utils import CalculationFactory
from aiida.work.run import submit
from aiida.orm.data.array.kpoints import KpointsData


In [ ]:
def setup_calc():
    kpoints = KpointsData()
    kpoints.set_kpoints_mesh([4,4,4])
    codename = 'pw-6.1@daint-s746'
    code = Code.get_from_string(codename)
    structure = StructureData(ase=atoms)
    options =  {'max_wallclock_seconds': 3600, 'resources': {'num_machines': 1}, 'queue_name':'low'}


    parameters = {
        'CONTROL': {
            'calculation': 'vc-relax',
            'restart_mode': 'from_scratch',
            'wf_collect': True,
            'scf_must_converge': False,
        },
        'SYSTEM': {
            'ecutwfc': 100.,
            'ecutrho': 400.,
        },
        'ELECTRONS': {
            'conv_thr': 1.e-6,
        },
    }

    inputs = {
        'code': code,
        'structure': structure,
        'pseudo': get_pseudos_from_structure(structure, 'SSSP_efficiency_v1.0'),
        'kpoints': kpoints,
        'parameters': ParameterData(dict=parameters),
        'settings': ParameterData(dict={}),
        '_options': options,
    }
    return inputs

In [ ]:
PwCalculation = CalculationFactory('quantumespresso.pw')

In [ ]:
def on_click_submit(b):
    with sumbmit_out:
        process = PwCalculation.process()
        calculation = submit(process, **setup_calc())
        print(calculation)

btn_submit = ipw.Button(description='Submit Cell Opt')
btn_submit.on_click(on_click_submit)
submit_out = ipw.Output()
display(btn_submit)